# Exploratory data analysis

In [ ]:
%matplotlib inline
from debruiteur.generator.datagenerator import DataGenerator
from debruiteur.preprocessing.preprocessor import *
from debruiteur.noise.noise import *
from debruiteur.utils.utils import plot_im_grid
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np

## Load data

In [ ]:
working_dir = os.path.abspath(os.getcwd())

df = make_dataframe()
df_resized = preprocess(df, os.path.join(working_dir, 'resized_images'))

## Noise

plot_im_grid(df, noise, rows, columns, figsize)

Parameters :
* df : dataframe, with first column containing image's path
* noise : Sublclass of Noise
* rows : Number of rows in image grid
* columns : Number of columns in image grid
* figsize : Tuple, shape of subfigure

### None

In [ ]:
plot_im_grid(df_resized)

### Gaussian

Parameters :
* mean
* std

In [ ]:
mu, sigma = 0, 10
plot_im_grid(df_resized, GaussianNoise(mean=mu, std=sigma))

### Poisson

No parameters

In [ ]:
plot_im_grid(df_resized, PoissonNoise())

### Uniform

Parameters :
* ampltitude : in ]0; 255[

In [ ]:
plot_im_grid(df_resized, UniformNoise(100))

### Salt and pepper noise

Parameters :
* p : probability in ]0;1[

In [ ]:
plot_im_grid(df_resized, SaltPepperNoise(0.3))

### Squared mask

Parameters :
* mask_shape : tuple, (width, height)
* target_freq : frequence in ]0; 1[

In [ ]:
plot_im_grid(df_resized, SquareMaskNoise(mask_shape=(10, 10), freq=0.1))

### Speckle

No parameters

In [ ]:
plot_im_grid(df_resized, SpeckleNoise())